# Lab4: Softmax And its Gradient

In [35]:
import numpy as np

def softmax(x: np.ndarray, 
            normalization: bool = True) -> np.ndarray:
    # first squeeze the array in case it is multi-dimentional
    x = np.squeeze(x)

    if len(x.shape) != 1:
        raise ValueError(f"The input is expected to be 1 dimensional. Found: {len(x.shape)} dimensions")

    # the main idea behind 'normalization' is to avoid numerical overflow with the softmax function 
    # (mainly with the denominator as a sum of exponential functions). The output of softmax for (x1, x2, ... xn) is the same as the output for
    # (x1 - C, x2 - C, ... xn - C) where C is any constant. (other operations such as division will alter the output)
    # consider the following link for the mathematical details : https://jaykmody.com/blog/stable-softmax/

    max_x = max(x) if normalization else 0
    x = x - max_x # if normalization is False, then 'x' is the same as the original output, otherwise the maximum element will be subtracted from each element
    sum_exp = np.sum(np.exp(x))
    return np.exp(x) / sum_exp

In [36]:
def softmax_grad1(x: np.ndarray, normalization: bool = True) -> np.ndarray:
    # the first step is to calculate the output of softmax on x
    s = softmax(x, normalization=normalization)
    s = np.expand_dims(s, axis=-1)
    assert s.shape == (len(x), 1)
    gradient =  - s @ np.transpose(s)
    # the current variable contains only - S_i . S_j for all i, j. the S_i term should be added to the diagonal entries
    for i in range(len(x)):
        gradient[i][i] += s[i][0]
    return gradient


# Lab5: Adding Backpropagation from Loss

## Softmax

In [37]:
def softmax_grad2(downstream_grad: np.ndarray, 
                  x: np.ndarray):
    """This function returns the gradient of the Loss with respect to the input 

    Args:
        downstream_grad (np.ndarray): the gradient of the loss with respect to the output of the softmax functions
        x (np.ndarray): input to the function
    """
    if len(downstream_grad.shape) > 2: 
        raise ValueError((f"The downstream gradient is expected to be at most {2} dimensional\n"
                          f"found {len(downstream_grad.shape)}"))
    if len(downstream_grad.shape) == 1:
        downstream_grad = np.expand_dims(downstream_grad, axis=-1)

    if len(downstream_grad) != len(x):
        raise ValueError((f"The function expects the input and the downstream grad to be of the same dimensions\n"
                          f"Found: input of length {len(x)} and downstream of dimension: {len(downstream_grad)}"))     

    # calculate the local gradient
    local_grad = softmax_grad1(x)
    # The entry e(i, j) in this matrix will represent the dS_i / d_xj
    
    # downstream_grad = [dL / dS_1, dL/d_S2, ... dL/dSn]
    # dL / d_k = sum of dL / dS_i * d_Si / d_xk
    # this can be simply computed with the usual matrix multiplication
    final_grad = local_grad @ downstream_grad
    return final_grad

In [38]:
x = np.array([1, 2, 3])
downgrad = np.array([-1, 2, 0])
print(softmax_grad2(downstream_grad=downgrad, x=x))

[[-0.12599116]
 [ 0.39170594]
 [-0.26571478]]


## Relu

In [39]:
def ReLU(x: np.ndarray):
    return x * (x > 0)

def ReLU_grad1(x: np.ndarray, jacobian_matrix: bool = True) -> np.ndarray:
    # the gradient of the relu function is quite simple:
    # if x > 0, it is 1, otherwise it is 0
    
    # nevertheless, the gradient can be expressed either a vector, or a Jacobian matrix (2d matrix)
    # since the i-th input x_i only affects the i-th output we have: all the non-digonal entries will be zero 
    if jacobian_matrix:
        grad = np.eye(N=len(x))
        for i, v in enumerate(x):
            grad[i][i] = float(v > 0)
        return grad
    return (x > 0).astype(float)


def ReLU_grad2(downstream_grad: np.ndarray, 
               x: np.ndarray) -> np.ndarray:
    # the main difference between Softmax and ReLU is that the i-th input x_i only affects the i-th output x_i
    # and thus 


    if len(downstream_grad.shape) > 2: 
        raise ValueError((f"The downstream gradient is expected to be at most {2} dimensional\n"
                          f"found {len(downstream_grad.shape)}"))
    if len(downstream_grad.shape) == 1:
        downstream_grad = np.expand_dims(downstream_grad, axis=-1)

    if len(downstream_grad) != len(x):
        raise ValueError((f"The function expects the input and the downstream grad to be of the same dimensions\n"
                          f"Found: input of length {len(x)} and downstream of dimension: {len(downstream_grad)}"))     

    # calculate the local gradient
    local_grad = ReLU_grad1(x, jacobian_matrix=True)
    final_grad = local_grad @ downstream_grad
    return final_grad    

In [40]:
# let's use the example from the lecture
x = np.array([1, -2, 3, -1])
downgrad = np.array([4, -1, 5, 9])
print(ReLU_grad2(downstream_grad=downgrad, x=x))
# the results match the ones in the 5-th lecture: 26-th slide

[[4.]
 [0.]
 [5.]
 [0.]]


## Linear Layer

In [49]:
# this function simply computes the 
from typing import List

def linear(x: np.ndarray,
           weight_matrix: np.ndarray):
    if len(x.shape) == 1:
        x = np.expand_dims(x, axis=-1)
    if len(x.shape) != 2 or x.shape[-1] != 1:
        raise ValueError(f"The input is expected to be a column vector: of the shape {'n', 1}\n"
                         f"Found: {x.shape}")
    # the weights are expected to be of the shape (None, x.shape[0])
    if len(weight_matrix.shape) != 2 and weight_matrix.shape[-1] != x.shape[0]:
        raise ValueError(f"The weight matrix second dimension and the length of the vector are expected to match")

    return weight_matrix @ x

def linear_grad1(x: np.ndarray, weight_matrix: np.ndarray) -> List[np.ndarray]:
    n, k1 = weight_matrix.shape
    k2, m = x.shape
    if k1 != k2:
        raise ValueError(f"The dimensions do not match")
    k = k1

    # the linear function accepts matrix input and generate matrix output
    # this function will return '|W|' matrices where each matrix represents the gradients o
    # of result with respect to w_ij

    def gradient_matrix(i, j):
        result = np.zeros(shape=(n, m), dtype=np.float32)
        # set the i-th row of the result matrix to the j-th column of the 'x' matrix 
        for index in range(m):
            result[i, index] = x[j, index] 
        return result
    
    grads = [gradient_matrix(i, j) for j in range(k1) for i in range(n)]
    return grads

def linear_grad2(downstream_grad, x: np.ndarray, weight_matrix: np.ndarray) -> np.ndarray:
    if len(x.shape) == 1:
        x = np.expand_dims(x, axis=-1)
    n, k1 = weight_matrix.shape
    k2, m = x.shape
    
    # make sure the downstream_grad is of the shape (n, m)
    if downstream_grad.shape != (n, m):
        raise ValueError("The downstream_grad must match the dimension of the output of the linear layer")

    local_grad = linear_grad1(x, weight_matrix)
    final_grad = np.zeros(shape=(n, k1))

    for i in range(n):
        for j in range(k1):
            final_grad[i, j] = np.sum(local_grad[j * k1 + i] * downstream_grad)
    return final_grad    

In [50]:
x = np.array([0.2, 0.4])
wm = np.array([[0.1, 0.5], [-0.3, 0.8]])
linear(x=x, weight_matrix=wm)
downgrad = np.array([[0.44], [0.52]])
linear_grad2(downstream_grad=downgrad, x=x, weight_matrix=wm)
# the results match the results in the 5-th lecture, 28-th slide.

array([[0.088, 0.176],
       [0.104, 0.208]])